In [ ]:
!pip install transformers datasets torch sentencepiece faiss-cpu accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import faiss
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    pipeline
)
from datasets import Dataset


In [ ]:
import pandas as pd

questions = pd.read_csv("/content/cleaned_questions.csv")
answers = pd.read_csv("/content/cleaned_answers.csv")

print(f"Number of questions: {len(questions)}")
print(f"Number of answers: {len(answers)}")


Number of questions: 6554
Number of answers: 659


In [ ]:
print("Missing values in questions:", questions.isnull().sum())
print("Missing values in answers:", answers.isnull().sum())


Missing values in questions: disease     0
question    0
label       0
dtype: int64
Missing values in answers: disease    0
label      0
answer     0
dtype: int64


In [ ]:
questions = questions.dropna()
answers = answers.dropna()


In [ ]:
questions = questions.reset_index(drop=True)
answers = answers.reset_index(drop=True)


In [ ]:
print(questions.head())
print(answers.head())


    disease                              question                label
0  diabetes                     what is diabetes?  diabetes definition
1  diabetes               tell me about diabetes?  diabetes definition
2  diabetes     what kind of disease is diabetes?  diabetes definition
3  diabetes        can you elaborate on diabetes?  diabetes definition
4  diabetes  what can you tell me about diabetes?  diabetes definition
    disease                   label  \
0  diabetes     diabetes definition   
1  diabetes       diabetes symptoms   
2  diabetes         diabetes causes   
3  diabetes          diabetes risks   
4  diabetes  diabetes complications   

                                              answer  
0  Diabetes mellitus is a metabolic disease that ...  
1  Diabetes symptoms are caused by rising blood s...  
2  Different causes are associated with each type...  
3  Certain factors increase your risk for diabete...  
4  High blood sugar damages organs and tissues th...  


In [ ]:
import pandas as pd

# Load datasets
questions = pd.read_csv("/content/cleaned_questions.csv")
answers = pd.read_csv("/content/cleaned_answers.csv")

# Print dataset sizes
print(f"Questions count: {len(questions)}")
print(f"Answers count: {len(answers)}")

# Check for NaN values
print("\nMissing values in Questions:", questions.isnull().sum().sum())
print("Missing values in Answers:", answers.isnull().sum().sum())


Questions count: 6554
Answers count: 659

Missing values in Questions: 0
Missing values in Answers: 0


In [ ]:
# Drop missing values & reset index
questions_clean = questions.dropna().reset_index(drop=True)
answers_clean = answers.dropna().reset_index(drop=True)

# Ensure sizes match after cleanup
min_size = min(len(questions_clean), len(answers_clean))
questions_clean = questions_clean.iloc[:min_size]
answers_clean = answers_clean.iloc[:min_size]

# Save fixed datasets
questions_clean.to_csv("/content/fixed_questions.csv", index=False)
answers_clean.to_csv("/content/fixed_answers.csv", index=False)

print("✅ Fixed dataset saved successfully!")


✅ Fixed dataset saved successfully!


In [ ]:
def load_custom_dataset():
    # Load cleaned dataset
    questions = pd.read_csv("/content/fixed_questions.csv")
    answers = pd.read_csv("/content/fixed_answers.csv")

    # Ensure final check
    if len(questions) != len(answers):
        raise ValueError("⚠️ Dataset size mismatch persists!")

    # Convert to Hugging Face Dataset
    dataset = Dataset.from_pandas(pd.DataFrame({
        "question": questions['question'].tolist(),
        "answer": answers['answer'].tolist()
    }))

    return dataset

dataset = load_custom_dataset()
print("✅ Dataset loaded successfully!")


✅ Dataset loaded successfully!


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load a sentence transformer model for embedding
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Load dataset
import pandas as pd

questions = pd.read_csv("/content/fixed_questions.csv")['question'].tolist()
answers = pd.read_csv("/content/fixed_answers.csv")['answer'].tolist()

# Convert questions into embeddings
question_embeddings = embedder.encode(questions, convert_to_numpy=True)

# Build FAISS index
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(question_embeddings))

print("✅ FAISS index built successfully!")

def retrieve_answer(user_query):
    query_embedding = embedder.encode([user_query], convert_to_numpy=True)
    D, I = index.search(np.array(query_embedding), k=1)  # Retrieve top 1 match
    best_match_idx = I[0][0]

    if D[0][0] > 100:  # If distance is too high, assume no good match
        return "I'm not sure. Let me generate a response for you!"

    return answers[best_match_idx]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS index built successfully!


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Use TinyLlama (small & open model)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
import os
print("Files in current directory:", os.listdir("."))


Files in current directory: ['.config', '.ipynb_checkpoints', 'fixed_questions.csv', 'fixed_answers.csv', 'cleaned_questions.csv', 'cleaned_answers.csv', 'sample_data']


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

# Load your datasets (Ensure these files exist)
questions_df = pd.read_csv("/content/cleaned_questions.csv")
answers_df = pd.read_csv("/content/cleaned_answers.csv")

# Load a Sentence Transformer model (You can use BioBERT too)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Generate embeddings for the medical questions
question_embeddings = model.encode(questions_df["question"].tolist(), convert_to_numpy=True)

# Save embeddings for future retrieval
np.save("medical_embeddings.npy", question_embeddings)

# Save answers for retrieval later
answers_df.to_csv("medical_answers.csv", index=False)

print("✅ Embeddings generated and saved!")


✅ Embeddings generated and saved!


In [ ]:
import faiss

# Load the saved embeddings
question_embeddings = np.load("medical_embeddings.npy")
dimension = question_embeddings.shape[1]

# Create a FAISS index for retrieval
index = faiss.IndexFlatL2(dimension)
index.add(question_embeddings)

# Save FAISS index
faiss.write_index(index, "medical_faiss.index")
print("✅ FAISS index created and saved!")


✅ FAISS index created and saved!


In [ ]:
import faiss
import numpy as np
import pandas as pd

# Load FAISS index
faiss_index = faiss.read_index("medical_faiss.index")

# Load question embeddings
question_embeddings = np.load("medical_embeddings.npy")

# Load answers dataset
answers_df = pd.read_csv("medical_answers.csv")

# Load model for encoding queries
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def retrieve_answer(query):
    """Retrieve the most relevant answer from FAISS index"""
    # Encode query
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Search in FAISS index
    _, idx = faiss_index.search(query_embedding, 1)  # Get the top-1 result

    # Retrieve the answer corresponding to the best-matching question
    retrieved_answer = answers_df.iloc[idx[0][0]]["answer"]

    return retrieved_answer

# Example usage
query = "What are the symptoms of diabetes?"
response = retrieve_answer(query)
print("Bot:", response)


Bot: A doctor or a mental health professional can diagnose MDD based on your symptoms, feelings, and behaviors.
Typically, you�ll be asked specific questions or given a questionnaire so health professionals can better determine whether you have MDD or another condition.
To be diagnosed with MDD, you need to meet the symptom criteria listed in the �Diagnostic and Statistical Manual of Mental Disorders, 5th edition (DSM-5).� This manual helps medical professionals diagnose mental health conditions.
According to its criteria:
You must experience a change in your previous functioning.
Symptoms must occur for a period of 2 or more weeks.
At least one symptom is either depressed mood or loss of interest or pleasure.
You must also experience five or more of the following symptoms in the 2-week period:
You feel sad or irritable most of the day, nearly every day.
You�re less interested in most activities you once enjoyed.
You suddenly lose or gain weight or have a change in appetite.
You have t

In [ ]:
import faiss
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load FAISS index
faiss_index = faiss.read_index("medical_faiss.index")

# Load answers dataset
answers_df = pd.read_csv("medical_answers.csv")

# Load model for encoding queries
retrieval_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Load a lightweight LLM (TinyLlama) for general medical queries
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tiny_llama = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
tiny_tokenizer = AutoTokenizer.from_pretrained(model_name)

def retrieve_answer(query):
    """Retrieve the most relevant answer from FAISS index"""
    query_embedding = retrieval_model.encode([query], convert_to_numpy=True)
    _, idx = faiss_index.search(query_embedding, 1)  # Get top-1 match

    # Ensure the index is within the valid range
    if idx[0][0] == -1 or idx[0][0] >= len(answers_df):
        return None  # No match found

    return answers_df.iloc[idx[0][0]]["answer"]

def generate_tiny_llama_response(query):
    """Generate a response using TinyLlama"""
    input_text = f"Answer this medical question: {query}"
    inputs = tiny_tokenizer(input_text, return_tensors="pt").to(device)

    outputs = tiny_llama.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9)
    return tiny_tokenizer.decode(outputs[0], skip_special_tokens=True)

def chatbot():
    """Main chatbot loop"""
    print("\n🤖 Medical Chatbot (Type 'exit' to stop)")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            print("👋 Exiting chatbot. Stay healthy!")
            break

        # Try to retrieve answer from FAISS
        response = retrieve_answer(user_input)

        # If no relevant answer found, use TinyLlama
        if response is None:
            print("🔄 No exact match found. Using TinyLlama to generate response...")
            response = generate_tiny_llama_response(user_input)

        print("Bot:", response)

# Run chatbot
chatbot()



🤖 Medical Chatbot (Type 'exit' to stop)

You: what is diabetes?
Bot: Diabetes mellitus is a metabolic disease that causes high blood sugar. Your body either doesn�t make enough insulin or can�t effectively use the insulin it makes.
The hormone insulin moves sugar from the blood into your cells to be stored or used for energy. If this malfunctions, you may have diabetes.
Untreated high blood sugar from diabetes can damage your nerves, eyes, kidneys, and other organs. But educating yourself about diabetes and taking steps to prevent or manage it can help you protect your health.

You: cause of asthma?
Bot: Diverticulitis can cause digestive symptoms and abdominal pain. Some people may develop complications that require urgent medical care.

You: symptoms of diabetes?
Bot: A doctor or a mental health professional can diagnose MDD based on your symptoms, feelings, and behaviors.
Typically, you�ll be asked specific questions or given a questionnaire so health professionals can better deter

In [ ]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load FAISS index
faiss_index = faiss.read_index("medical_faiss.index")

# Load answers dataset
answers_df = pd.read_csv("medical_answers.csv")

# Load Sentence Transformer model for encoding queries
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def retrieve_answer(query):
    """Retrieve the most relevant answer from FAISS index"""
    # Encode query
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Search in FAISS index
    _, idx = faiss_index.search(query_embedding, 1)  # Get the top-1 result

    # Retrieve the answer corresponding to the best-matching question
    retrieved_answer = answers_df.iloc[idx[0][0]]["answer"]

    return retrieved_answer

def chatbot():
    """Interactive chatbot loop"""
    print("\n🤖 Medical Chatbot (Type 'exit' to stop)")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            print("👋 Exiting chatbot. Stay healthy!")
            break

        response = retrieve_answer(user_input)
        print("Bot:", response)

# Run chatbot
chatbot()



🤖 Medical Chatbot (Type 'exit' to stop)

You: what is asthma?
Bot: Low blood sugar can occur for a number of reasons. It�s usually a side effect of diabetes treatment.
Possible causes in people with diabetes
If you have diabetes, your blood sugar levels may sometimes become too low as a result of your treatment. Hypoglycemia may also occur if you:
take too much medication, including insulin or medications that increase insulin production
plan to eat a big meal and take your medication accordingly but then do not eat enough
skip meals or fast
eat less than usual
eat later than usual but take your medication at the usual time
engage in unplanned, excessive physical activity without eating enough
drink alcohol, which can affect how your body manages your blood sugar levels
become sick and cannot keep food down
Blood sugar regulation
Blood sugar regulation aims to prevent spikes and drops by keeping your blood sugar levels stable.
If you have untreated diabetes or are unaware that you hav